# Tempo
Testing librosa tempo detection

In [294]:
import numpy as np
from matplotlib import pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import math
import glob

import warnings
warnings.filterwarnings("ignore")

Let's load a snippet of a song.

In [295]:

# SRC_PATH = '**/*Ed*'
# START = 60.34
# # LEN = 5.0
# # LEN = 10.0
# # LEN = 2.0
# LEN = 20.0
# KNOWN_TEMPO = 126

# SRC_PATH = '**/*126 BPM*'
# START = 22.8
# LEN = 5
# KNOWN_TEMPO = 126

SRC_PATH = '**/*Dua*Rules*'
# START = 60.8
# LEN = 5
# LEN = 10
START = 80.8
LEN = 20
KNOWN_TEMPO = 116

END = START + LEN

src = glob.glob(SRC_PATH)[0]
y, sr = librosa.load(src, offset=START, duration=LEN)
ipd.Audio(y, rate=sr)

Now use built in beat track method, and overlay clicks onto detected segment. We often hear a decent tempo bpm estimation, and phase / offset is usually good enough (often up beat instead of down however, the "ANDs" of "1 AND 2 AND 3 4 AND ..." instead of 1, 2, 3, 4, ...). This isn't super important for LED displays?

In [296]:
onset_env = librosa.onset.onset_strength(y=y, sr=sr, aggregate=np.median)
tempo, beats = librosa.beat.beat_track(onset_envelope=onset_env, start_bpm=126,
                                       tightness=800,
                                       trim=False,
                                       sr=sr, units='time')
ipd.display('Estimated tempo {0}, num beats detected {1} vs (not quite right...) expected {2}'.format(
    tempo, len(beats), math.floor(KNOWN_TEMPO * LEN / 60.0)))
ipd.display(beats)
ipd.display(np.diff(beats))

click_track = librosa.clicks(times=beats, sr=sr, length=len(y))
ipd.Audio(y + click_track, rate=sr)


'Estimated tempo 151.99908088235293, num beats detected 25 vs (not quite right...) expected 19'

array([0.16253968, 0.55727891, 0.95201814, 1.34675737, 1.7414966 ,
       2.13623583, 2.5077551 , 2.90249433, 3.29723356, 3.66875283,
       4.04027211, 4.43501134, 4.82975057, 5.2244898 , 5.61922902,
       6.01396825, 6.40870748, 6.80344671, 7.19818594, 7.59292517,
       7.9876644 , 8.38240363, 8.77714286, 9.17188209, 9.58984127])

array([0.39473923, 0.39473923, 0.39473923, 0.39473923, 0.39473923,
       0.37151927, 0.39473923, 0.39473923, 0.37151927, 0.37151927,
       0.39473923, 0.39473923, 0.39473923, 0.39473923, 0.39473923,
       0.39473923, 0.39473923, 0.39473923, 0.39473923, 0.39473923,
       0.39473923, 0.39473923, 0.39473923, 0.41795918])

Curiously, the reported tempo from Librosa is often not as good as the one indicated by the detected beats.

In [297]:
reported_tempo = tempo
spb = np.average(np.diff(beats)) # beats per sec
tempo = 60 / spb

ipd.display('Reported tempo {} vs derrived {} vs known {}, spb {}'.format(reported_tempo, tempo, KNOWN_TEMPO, spb))

'Reported tempo 151.99908088235293 vs derrived 152.74784482758622 vs known 116, spb 0.39280423280423277'

Now let's use that prediction and overlay the would be assumed beats onto the next chunk of the track and see how it sounds.

In [298]:
# never got this right...

doubled, _ = librosa.load(src, offset=START, duration=2.0 * LEN)
shift = LEN
# shift = END - beats[-1] + LEN - START
# shift = LEN + LEN - beats[-1]
shifted_beats = beats + shift
# ipd.display(beats[-2], beats[-1], beats[0] + shift, beats[1] + shift)
extra = np.array([])

# if it seems like we'd be missing one, let's stick it in here
if ((LEN + beats[0] - beats[-1]) / (2 * spb) > 0.85):
    extra = np.array([ (LEN + beats[0] + beats[-1]) / 2.0 ])
ipd.display(beats[-1], extra, beats[0] + LEN)
doubled_click = librosa.clicks(times=np.concatenate((beats, extra, shifted_beats)), sr=sr, length=len(doubled))
ipd.Audio(doubled + doubled_click, rate=sr)

9.58984126984127

array([], dtype=float64)

10.162539682539682

Or choose just one beat and use the derived tempo to create overlayed click track.

In [299]:
times = [beats[0]]
while times[-1] < END:
    times = times + [ times[-1] + spb ]
ipd.Audio(doubled + librosa.clicks(times=times, sr=sr, length=len(doubled)), rate=sr)

This might be good enough?

My shift logic isn't quite right. But assuming it was, sometimes one sounds better, or they're often close. It may also be that longer windows get derived (not reported!) tempo closer to known. Maybe I can use the derived to find the best / a good "anchor" time, and listen clicks around it...